In [16]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 166.7 MB 3.7 kB/s  eta 0:00:01  |█▏                              | 5.9 MB 3.7 MB/s eta 0:00:44     |█▊                              | 9.0 MB 3.7 MB/s eta 0:00:44     |█▉                              | 9.6 MB 3.7 MB/s eta 0:00:43     |██                              | 11.0 MB 3.7 MB/s eta 0:00:43     |██▉                             | 14.8 MB 3.7 MB/s eta 0:00:42     |████▏                           | 21.8 MB 25.1 MB/s eta 0:00:06     |████▎                           | 22.5 MB 25.1 MB/s eta 0:00:06     |█████                           | 26.1 MB 25.1 MB/s eta 0:00:06     |█████▎                          | 27.2 MB 25.1 MB/s eta 0:00:06     |█████▍                          | 28.2 MB 25.1 MB/s eta 0:00:06     |█████▍                          | 28.2 MB 25.1 MB/s eta 0:00:06     |█████▌                          | 28.6 MB 25.1 MB/s eta 0:00:06     |█████▌                       

In [1]:
pip install xgboost pandas sklearn

Defaulting to user installation because normal site-packages is not writeable
  Using cached xgboost-1.4.2-py3-none-manylinux2010_x86_64.whl (166.7 MB)
  Using cached sklearn-0.0-py2.py3-none-any.whl
You should consider upgrading via the '/home/daiecasquero/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle, islice
import datetime as datetime
import glob
from matplotlib import pyplot
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
import warnings
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression as LR
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold

%matplotlib inline

plt.style.use('default')
sns.set(style="whitegrid")
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (15, 5)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from catboost import CatBoostRegressor

In [13]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler
# the model
from sklearn.ensemble import RandomForestClassifier
# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline
# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [14]:
train_labels = pd.read_csv('train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
train_values = pd.read_csv('train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [15]:
train = train_values.merge(train_labels, on='building_id')
train

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,2
669485,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
602512,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,3


In [16]:
train = train.drop('geo_level_1_id', axis=1)
train = train.drop('geo_level_2_id', axis=1)
train = train.drop('geo_level_3_id', axis=1)
train = train.drop('land_surface_condition', axis=1)
#train = train.drop('count_floors_pre_eq', axis=1)

In [17]:
train = train.drop('roof_type', axis=1)
train = train.drop('other_floor_type', axis=1)
train = train.drop('position', axis=1)
#train = train.drop('other_floor_type', axis=1)
train = train.drop('plan_configuration', axis=1)
train = train.drop('legal_ownership_status', axis=1)
train = train.drop('ground_floor_type', axis=1)
train = train.drop('foundation_type', axis=1)

In [18]:
train_value = train['damage_grade']
train_data = train.iloc[:, train.columns != 'damage_grade']

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(train_data, train_value, train_size=0.7, random_state=1234)

In [20]:
modelo = xgb.XGBClassifier(objective='reg:logistic', n_estimators=10, seed=123, learning_rate=0.7, 
                           max_depth=4, alpha=30)

In [10]:
modelo

XGBClassifier(alpha=30, base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=0.7, max_delta_step=None, max_depth=4,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=10, n_jobs=None, num_parallel_tree=None,
              objective='reg:logistic', random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, seed=123, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)

In [11]:
modelo.fit(X_train, y_train)

# Realizo las predicciones
y_pred = modelo.predict(X_train)
predicciones = [round(value) for value in y_pred]
# Evalúo las predicciones
precision_train = accuracy_score(y_train, predicciones)

# Repito el proceso con datos de evaluacion
y_pred = modelo.predict(X_validation)
predicciones = [round(value) for value in y_pred]

# Evalúo las predicciones
precision_test = accuracy_score(y_validation, predicciones)
print(modelo)
print('Precisión xgboost train/test  {0:.3f}/{1:.3f}'
      .format(precision_train, precision_test))

[21:27:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier(alpha=30, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.7, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=10, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=123, reg_alpha=30,
              reg_lambda=1, scale_pos_weight=None, seed=123, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
Precisión xgboost train/test  0.590/0.594


In [21]:
params = {
        'gamma': [0.5, 1, 1.5, 2, 5],
        'max_depth': [3, 4, 5, 8, 10],
        'n_estimators': [5, 10, 20, 50],
        'learning_rate': [0.1, 0.3, 0.5, 0.7, 0.9],
        'alpha': [0, 5, 15, 30]
        }

In [22]:
folds = 3
param_comb = 500

In [23]:
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

In [24]:
random_search = RandomizedSearchCV(modelo, param_distributions=params, 
        n_iter=param_comb, n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=1001)

In [25]:
random_search

RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x0000020EC72A1740>,
                   estimator=XGBClassifier(alpha=30, base_score=None,
                                           booster=None, colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.7,
                                           max_delta_step=None, max_depth=4,
                                           min_child_weight=Non...
                                           random_state=None, reg_alpha=None,
                                           reg_lambda=None,
                                           scale_pos_weight=None, seed=123,
                                           subsampl

In [26]:
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  3.3min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 10.8min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed: 24.8min
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed: 41.5min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 67.9min
[Parallel(n_jobs=4)]: Done 1144 tasks      | elapsed: 96.9min
[Parallel(n_jobs=4)]: Done 1500 out of 1500 | elapsed: 127.2min finished


[23:46:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x0000020EC72A1740>,
                   estimator=XGBClassifier(alpha=30, base_score=None,
                                           booster=None, colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.7,
                                           max_delta_step=None, max_depth=4,
                                           min_child_weight=Non...
                                           random_state=None, reg_alpha=None,
                                           reg_lambda=None,
                                           scale_pos_weight=None, seed=123,
                                           subsampl

In [28]:
predic = random_search.predict_proba(X_validation)

In [29]:
predic

array([[0.04307372, 0.59681815, 0.3601081 ],
       [0.43316516, 0.5439617 , 0.02287317],
       [0.04620108, 0.56988406, 0.38391486],
       ...,
       [0.07585122, 0.58564115, 0.33850768],
       [0.02114557, 0.58264846, 0.39620596],
       [0.0545476 , 0.54568654, 0.3997659 ]], dtype=float32)

In [27]:
print(random_search.best_estimator_)

XGBClassifier(alpha=0, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=123, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=123, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [30]:
# Realizo las predicciones
y_pred = random_search.predict(X_train)
predicciones = [round(value) for value in y_pred]
# Evalúo las predicciones
precision_train = accuracy_score(y_train, predicciones)

# Repito el proceso con datos de evaluacion
y_pred = random_search.predict(X_validation)
predicciones = [round(value) for value in y_pred]

# Evalúo las predicciones
precision_test = accuracy_score(y_validation, predicciones)
print(random_search)
print('Precisión xgboost train/test  {0:.3f}/{1:.3f}'
      .format(precision_train, precision_test))

RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x0000020EC72A1740>,
                   estimator=XGBClassifier(alpha=30, base_score=None,
                                           booster=None, colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.7,
                                           max_delta_step=None, max_depth=4,
                                           min_child_weight=Non...
                                           random_state=None, reg_alpha=None,
                                           reg_lambda=None,
                                           scale_pos_weight=None, seed=123,
                                           subsampl

In [191]:
print(random_search.cv_results_)

{'mean_fit_time': array([ 9.62469101, 10.49547577,  8.13808227,  8.11446516, 21.80566208,
       10.61449067, 13.26746043,  8.03217371, 10.58706681, 16.67055289]), 'std_fit_time': array([0.04523287, 0.08111174, 0.01717539, 0.04314762, 0.11047848,
       0.1382433 , 0.08456394, 0.00874173, 0.04972601, 1.27990643]), 'mean_score_time': array([0.17114194, 0.17057284, 0.14982057, 0.14905532, 0.26392182,
       0.19270388, 0.19468212, 0.13495223, 0.16880989, 0.1649607 ]), 'std_score_time': array([0.01838932, 0.01797345, 0.00926216, 0.00248485, 0.00846168,
       0.00623902, 0.00545539, 0.00821837, 0.00874689, 0.04133804]), 'param_max_depth': masked_array(data=[5, 4, 3, 3, 8, 4, 5, 3, 4, 8],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[2, 1.5, 5, 1.5, 5, 2, 0.5, 0.5, 0.5, 2],
             mask=[False, False, False, False, False, False, False, Fals

In [84]:
len(predicciones)

78181

In [192]:
test = pd.read_csv('test_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')

pd.options.display.float_format = '{:20,.2f}'.format

In [193]:
test = test.drop('geo_level_1_id', axis=1)
test = test.drop('geo_level_2_id', axis=1)
test = test.drop('geo_level_3_id', axis=1)
test = test.drop('land_surface_condition', axis=1)
#test = test.drop('count_floors_pre_eq', axis=1)

In [194]:
test = test.drop('roof_type', axis=1)
test = test.drop('other_floor_type', axis=1)
test = test.drop('position', axis=1)
#test = test.drop('other_floor_type', axis=1)
test = test.drop('plan_configuration', axis=1)
test = test.drop('legal_ownership_status', axis=1)
test = test.drop('ground_floor_type', axis=1)
test = test.drop('foundation_type', axis=1)

In [195]:
predic = random_search.predict_proba(test)

In [196]:
predic

array([[0.01848812, 0.44088084, 0.54063106],
       [0.02994292, 0.6846277 , 0.28542936],
       [0.05369101, 0.54561335, 0.40069568],
       ...,
       [0.0606572 , 0.62307173, 0.31627104],
       [0.04825808, 0.6449669 , 0.306775  ],
       [0.34072724, 0.6231002 , 0.03617253]], dtype=float32)

In [197]:
results = []
for line in predic:
    if line[0] > line [1] and line [0] > line [2]:
        results.append(1)
    elif line[1] > line[2]:
        results.append(2)
    else:
        results.append(3)

In [198]:
results

[3,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 1,
 3,
 2,
 2,
 3,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [199]:
damages = pd.Series(results).rename("damage_grade")

In [200]:
damages

0        3
1        2
2        2
3        1
4        2
        ..
86863    2
86864    3
86865    2
86866    2
86867    2
Name: damage_grade, Length: 86868, dtype: int64

In [202]:
ids = test.reset_index()['building_id']

In [203]:
ids

0         300051
1          99355
2         890251
3         745817
4         421793
          ...   
86863     310028
86864     663567
86865    1049160
86866     442785
86867     501372
Name: building_id, Length: 86868, dtype: int64

In [204]:
submit = pd.merge(ids, damages, left_index=True,right_index=True).set_index('building_id')

In [205]:
submit

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,2
...,...
310028,2
663567,3
1049160,2


In [ ]:
modelo = xgb.XGBClassifier()
modelo.fit(X_train, y_train)

# Realizo las predicciones
y_pred = modelo.predict(test)
predicciones = [round(value) for value in y_pred]
# Evalúo las predicciones
precision_train = accuracy_score(y_train, predicciones)
print(modelo)
print('Precisión xgboost train/test  {0:.3f}/{1:.3f}'
      .format(precision_train, precision_test))


In [ ]:
ids = test.reset_index()['building_id']

In [89]:
ids

0         300051
1          99355
2         890251
3         745817
4         421793
          ...   
86863     310028
86864     663567
86865    1049160
86866     442785
86867     501372
Name: building_id, Length: 86868, dtype: int64

In [90]:
damages = pd.Series(predicciones).rename("damage_grade")

In [91]:
damages

0        3
1        2
2        2
3        1
4        3
        ..
86863    2
86864    3
86865    2
86866    2
86867    2
Name: damage_grade, Length: 86868, dtype: int64

In [206]:
submit.to_csv('submit_xgboost_1.csv')